In [37]:
import os
os.chdir("/home/canyon/hackday-q2-2024-open-earth-foundation")
from src.utils import *
import censusdis.data as ced
import matplotlib.pyplot as plt
alber_eq_us = "EPSG:5070"

In [38]:
import censusdis.data as ced

DATASET = "acs/acs5"
YEAR = 2020
CENSUS_VARS = {"NAME" : "NAME",
               "B01001_001E" : "total_pop",
               "B25003_001E": "total_households",
               "B08201_002E" : "households_no_vehicle",

}

us_state_pops = ced.download(DATASET, YEAR, CENSUS_VARS, state= "*", with_geometry=True).rename(CENSUS_VARS, axis = 1)
us_state_pops["vehicle_households"] = us_state_pops["total_households"] - us_state_pops["households_no_vehicle"]
us_state_pops.head(3)


,STATE,NAME,total_pop,total_households,households_no_vehicle,geometry,vehicle_households
0,42,Pennsylvania,12794885,5106601,544554,"POLYGON ((-80.51989 40.90666, -80.51963 40.911...",4562047
1,06,California,39346023,13103114,920362,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",12182752
2,54,West Virginia,1807426,734235,62082,"POLYGON ((-82.64320 38.16909, -82.64300 38.169...",672153


In [39]:
us_city_pops = load_data_from_ts3("City-Level-Data/city_pops_full.geojson")

In [40]:
us_cities_state = us_city_pops.merge(us_state_pops[["STATE", "NAME", "total_pop", "vehicle_households"]], left_on = "fip", right_on = "STATE")

In [41]:
us_cities_state["pop_prop"] = us_cities_state["interpolated_pop"] / us_cities_state["total_pop"]
us_cities_state["vehicle_household_prop"] = us_cities_state["interpolated_households_with_vehicle"] / us_cities_state["vehicle_households"]

In [51]:
city_trip_distance = load_data_from_ts3("BTS Trips/city_trip_distances.csv")
state_trip_distance = load_data_from_ts3("BTS Trips/state_trip_distances.csv")
state_trip_distance["STATE"] = state_trip_distance["STATE"].astype(str).str.zfill(2)

In [55]:
us_cities_state = us_cities_state.merge(city_trip_distance).merge(state_trip_distance)
us_cities_state["trip_distance_prop"] = us_cities_state["interpolated_tract_miles_traveled"] / us_cities_state["total_miles_state"]

In [63]:
city_weights = us_cities_state[["place_id", "NAME", "name", "state", "fip", "interpolated_pop", "pop_prop", "vehicle_household_prop", "trip_distance_prop"]]
city_weights.sample(3)

,place_id,NAME,name,state,fip,interpolated_pop,pop_prop,vehicle_household_prop,trip_distance_prop
7087,297666915,Virginia,Tappahannock,Virginia,51,360.046957,0.000042,0.000053,0.000073
1264,297062345,Georgia,Bethlehem,Georgia,13,717.516111,0.000068,0.000063,0.000074
7885,297769020,Maryland,Laurel,Maryland,24,21707.173415,0.003595,0.003753,0.003266


In [64]:
state_fuel = load_data_from_ts3("DOT Fuel Sales/state_sales_23.csv")
state_fuel.head(3)

,state,total_fuel
0,Alabama,4209782982
1,Alaska,502883277
2,Arizona,4086269823


In [65]:
us_cities_state_fuel = state_fuel.merge(us_cities_state, left_on = "state", right_on = "state")
us_cities_state_fuel["fuel_weighted_pop"] = us_cities_state_fuel["total_fuel"] * us_cities_state_fuel["pop_prop"]
us_cities_state_fuel["fuel_weighted_vehicle_households"] = us_cities_state_fuel["total_fuel"] * us_cities_state_fuel["vehicle_household_prop"]
us_cities_state_fuel["fuel_weighted_trip_dist"] = us_cities_state_fuel["total_fuel"] * us_cities_state_fuel["trip_distance_prop"]

In [67]:
us_cities_state_fuel.query("interpolated_pop > 100000")

,state,total_fuel,id,place_id,addresstype,type,name,fip,interpolated_pop,interpolated_households,...,total_pop,vehicle_households,pop_prop,vehicle_household_prop,interpolated_tract_miles_traveled,total_miles_state,trip_distance_prop,fuel_weighted_pop,fuel_weighted_vehicle_households,fuel_weighted_trip_dist
98,Alabama,4209782982,4409,297327940,city,administrative,Montgomery,01,186830.148638,74665.667810,...,4893186,1776655,0.038182,0.038395,1.067587e+10,3.339506e+11,0.031968,1.607367e+08,1.616351e+08,1.345801e+08
129,Alabama,4209782982,6483,297583092,city,administrative,Huntsville,01,193265.021611,83388.311498,...,4893186,1776655,0.039497,0.044334,1.010963e+10,3.339506e+11,0.030273,1.662728e+08,1.866364e+08,1.274420e+08
170,Alabama,4209782982,9167,297861758,city,administrative,Birmingham,01,215569.520765,91951.355912,...,4893186,1776655,0.044055,0.044873,1.272924e+10,3.339506e+11,0.038117,1.854622e+08,1.889074e+08,1.604649e+08
191,Alabama,4209782982,11156,297978623,city,administrative,Mobile,01,185953.775530,76604.740297,...,4893186,1776655,0.038003,0.039175,1.183833e+10,3.339506e+11,0.035449,1.599827e+08,1.649172e+08,1.492340e+08
484,Arizona,4086269823,657,296938306,city,administrative,Gilbert,04,239151.805456,76799.483472,...,7174064,2489436,0.033336,0.030373,1.343884e+10,4.396352e+11,0.030568,1.362183e+08,1.241134e+08,1.249097e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14191,Washington,3396178526,12708,298511481,city,administrative,Tacoma,53,214189.583146,86131.958618,...,7512465,2705125,0.028511,0.029041,1.269227e+10,4.430747e+11,0.028646,9.682921e+07,9.862963e+07,9.728656e+07
14200,Washington,3396178526,14710,369415546,city,administrative,Spokane,53,210601.369075,89400.194667,...,7512465,2705125,0.028034,0.029594,1.309654e+10,4.430747e+11,0.029558,9.520708e+07,1.005075e+08,1.003853e+08
14236,Wisconsin,3601455189,2142,297117778,city,administrative,Milwaukee,55,592217.404002,229053.996284,...,5806975,2221191,0.101984,0.085438,3.164459e+10,4.421892e+11,0.071563,3.672901e+08,3.077022e+08,2.577327e+08
14375,Wisconsin,3601455189,6075,297512406,city,administrative,Green Bay,55,103358.302911,42406.435715,...,5806975,2221191,0.017799,0.017553,7.416309e+09,4.421892e+11,0.016772,6.410227e+07,6.321696e+07,6.040289e+07
